In [106]:
client_id = '3ywXGq4PyUoR_XXDvK_RIA'
api_key = 'FPOb0CFdwPxb8_4qN-r89V7n_74JWBe-xwT2zMhRXpd0oCxsn22aY0aITmZZo7FGZvzeKyYMlu7P4C72DXyEAxqpB3_A9aPRjJH1RBXkkwnsoilWCjJSwm-N-Vj2W3Yx'

In [3]:
bookmark_url = 'https://www.yelp.com/user_details_bookmarks?userid='
review_url = 'https://www.yelp.com/user_details_reviews_self?userid='
friends_url = 'https://www.yelp.com/user_details_friends?userid='
self_id = 'ShHBKjuJbQAVBLs7DgA95A'

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pickle
import json

from sklearn.metrics import jaccard_similarity_score
from scipy.stats import pearsonr

In [5]:
def save_obj(obj, name ):
    """ save object """
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    """ load object """
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [6]:
def split_biz_url(url):
    """ helper function to get the unique business name 
        by splitting /biz/ out from scraped data"""
    try:
        return url.split('/biz/')[1]       
    except:
        pass

In [109]:
def create_df_bookmarks(key, bookmarks):
    """ helper function to create dataframe of bookmarks from friends scraped data for list of dataframes"""
    df = pd.DataFrame(bookmarks[key], columns=['url'])
    df['id'] = key
    df['business_details'] = df['url'].apply(lambda x: split_biz_url(x))
    return df

In [8]:
def create_df_reviews(key):
    """ create dataframe of reviews from scraped data"""
    df = pd.DataFrame(list(all_reviews[key].items()), columns=['business', 'rating'])
    df['id'] = key
    return df

In [105]:
def get_biz_details(api_key, business):
    """ call business details endpoint of yelp api and returns a json object"""
    try:
        url = 'https://api.yelp.com/v3/businesses/{}'.format(business)
        headers = {'Authorization' : 'Bearer %s' % api_key}
        resp = requests.get(url=url, headers=headers)
        return resp.json()
    except:
        pass

In [79]:
def json_to_df(business, details):
    """ old function - to delete"""
    df_dict = {det: [business[det]] for det in details}
    cats = {'cat_{}'.format(i):[business['categories'][i]['alias']] for i in range(len(business['categories']))}
    df_dict.update(cats)
    return pd.DataFrame(df_dict)

In [84]:
def json_to_df(business, details):
    """ create dataframe from json object after calling api endpoint for specific details """
    df_dict = {}
    try:
        for det in details:
            df_dict[det] = business[det]
    except:
        pass
    cats = {'cat_{}'.format(i):[business['categories'][i]['alias']] for i in range(len(business['categories']))}
    df_dict.update(cats)
    
    return pd.DataFrame(df_dict)

In [107]:
def api_to_df(api_key, details, df):
    """ uses the json_to_df and get_biz_details functions to
            first call business details endpoint from yelp api
            then returns a dataframe based on details desired
    """
    api_call = []

    for business in df['business_id']:
        biz_details = get_biz_details(api_key, business)
        api_call.append(biz_details)    
    
    df_temp = []

    for business in api_call:
        df_temp.append(json_to_df(business, details))
    
    biz_details_df = pd.concat(df_temp)
    
    return biz_details_df

# Test

In [15]:
test = get_biz_details(api_key, my_bm_df['business_id'][0])
test

{'alias': 'three-twins-ice-cream-san-francisco-5',
 'categories': [{'alias': 'icecream', 'title': 'Ice Cream & Frozen Yogurt'},
  {'alias': 'desserts', 'title': 'Desserts'}],
 'coordinates': {'latitude': 37.772159, 'longitude': -122.430252},
 'display_phone': '(415) 487-8946',
 'hours': [{'hours_type': 'REGULAR',
   'is_open_now': False,
   'open': [{'day': 0, 'end': '2200', 'is_overnight': False, 'start': '1500'},
    {'day': 1, 'end': '2200', 'is_overnight': False, 'start': '1400'},
    {'day': 2, 'end': '2200', 'is_overnight': False, 'start': '1400'},
    {'day': 3, 'end': '2200', 'is_overnight': False, 'start': '1400'},
    {'day': 4, 'end': '2300', 'is_overnight': False, 'start': '1400'},
    {'day': 5, 'end': '2300', 'is_overnight': False, 'start': '1300'},
    {'day': 6, 'end': '2200', 'is_overnight': False, 'start': '1300'}]}],
 'id': 'ifMcMlpEtexz-yiSdcsQQg',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/8LlHcdd6M71NSepB_jCKsQ/o.jpg',
 'is_claimed': True,
 'is_closed'

In [47]:
test

{'alias': 'three-twins-ice-cream-san-francisco-5',
 'categories': [{'alias': 'icecream', 'title': 'Ice Cream & Frozen Yogurt'},
  {'alias': 'desserts', 'title': 'Desserts'}],
 'coordinates': {'latitude': 37.772159, 'longitude': -122.430252},
 'display_phone': '(415) 487-8946',
 'hours': [{'hours_type': 'REGULAR',
   'is_open_now': False,
   'open': [{'day': 0, 'end': '2200', 'is_overnight': False, 'start': '1500'},
    {'day': 1, 'end': '2200', 'is_overnight': False, 'start': '1400'},
    {'day': 2, 'end': '2200', 'is_overnight': False, 'start': '1400'},
    {'day': 3, 'end': '2200', 'is_overnight': False, 'start': '1400'},
    {'day': 4, 'end': '2300', 'is_overnight': False, 'start': '1400'},
    {'day': 5, 'end': '2300', 'is_overnight': False, 'start': '1300'},
    {'day': 6, 'end': '2200', 'is_overnight': False, 'start': '1300'}]}],
 'id': 'ifMcMlpEtexz-yiSdcsQQg',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/8LlHcdd6M71NSepB_jCKsQ/o.jpg',
 'is_claimed': True,
 'is_closed'

In [51]:
details = ['alias', 'is_closed', 'price', 'rating', 'review_count']

In [41]:
for i in range(len(test['categories'])):
    list(test['categories'][i]['alias'])

In [80]:
json_to_df(test, details)

,alias,cat_0,cat_1,is_closed,price,rating,review_count
0,three-twins-ice-cream-san-francisco-5,icecream,desserts,True,$,4.0,149


In [20]:
app_id = 'client_id'
app_secret = 'client_secret'
data = {'grant_type': 'client_credentials',
        'client_id': app_id,
        'client_secret': app_secret}
url = 'https://api.yelp.com/v3/businesses/{}'.format("maria-catita-restaurante-e-loja-regional-lisboa")
headers = {'Authorization' : 'Bearer %s' % client_secret}
resp =requests.get(url=url, headers=headers)
print(resp.status_code)

200


In [31]:
json_resp = resp.json()
print(json.dumps(json_resp, sort_keys = True, indent = 4))

{
    "alias": "maria-catita-restaurante-e-loja-regional-lisboa",
    "categories": [
        {
            "alias": "portuguese",
            "title": "Portuguese"
        }
    ],
    "coordinates": {
        "latitude": 38.709286,
        "longitude": -9.1343429
    },
    "display_phone": "+351 21 133 1313",
    "hours": [
        {
            "hours_type": "REGULAR",
            "is_open_now": true,
            "open": [
                {
                    "day": 0,
                    "end": "0000",
                    "is_overnight": false,
                    "start": "1200"
                },
                {
                    "day": 1,
                    "end": "0000",
                    "is_overnight": false,
                    "start": "1200"
                },
                {
                    "day": 2,
                    "end": "0000",
                    "is_overnight": false,
                    "start": "1200"
                },
                {
        

In [34]:
json_resp['alias']

'maria-catita-restaurante-e-loja-regional-lisboa'

In [24]:
print(resp.headers)

{'Server': 'nginx', 'Content-Type': 'application/json', 'RateLimit-DailyLimit': '5000', 'RateLimit-Remaining': '4997', 'RateLimit-ResetTime': '2018-11-23T00:00:00+00:00', 'Content-Encoding': 'gzip', 'X-Proxied': '10-69-90-16-uswest2aprod', 'Transfer-Encoding': 'chunked', 'Accept-Ranges': 'bytes', 'Date': 'Thu, 22 Nov 2018 21:12:01 GMT', 'Via': '1.1 varnish', 'Connection': 'keep-alive', 'X-Served-By': 'cache-pao17427-PAO', 'X-Cache': 'MISS', 'X-Cache-Hits': '0'}


# Get business details from bookmarks

https://www.yelp.com/developers/documentation/v3/business

| Name | Type   |  Description   |
|------|------|------|
|   categories | object[]| A list of category title and alias pairs associated with this business.|
|   categories[x].alias | string| Alias of a category, when searching for business in certain categories, use alias rather than the title.|
|   alias | string| Unique Yelp alias of this business. Can contain unicode characters. Example: 'yelp-san-francisco'. Also see: What's the difference between the Yelp business ID and business alias?|
|   is_closed | bool| Whether business has been (permanently) closed|
|   price | string| Price level of the business|
|   rating | decimal| Rating for this business|
|   review_count | int| Number of reviews for this business.|

In [11]:
self_bookmarks = load_obj('self_bookmarks')
friend_bookmarks = load_obj('friend_bookmarks')

In [12]:
self_bookmarks

['/biz/three-twins-ice-cream-san-francisco-5',
 '/biz/ben-thai-cafe-san-francisco',
 '/biz/ijji-sushi-san-francisco',
 '/biz/koi-palace-daly-city',
 '/biz/robin-san-francisco',
 '/biz/maria-catita-restaurante-e-loja-regional-lisboa',
 '/biz/juns-beauty-salon-san-francisco',
 '/biz/bowld-acai-san-francisco-2',
 '/biz/china-bee-san-mateo',
 '/biz/bobos-san-francisco-14',
 '/biz/grand-lake-kitchen-oakland',
 '/biz/nabe-san-francisco-5',
 '/biz/dumpling-kitchen-san-francisco',
 '/biz/nojo-ramen-tavern-san-francisco-2',
 '/biz/el-milagro-los-angeles-2',
 '/biz/five-happiness-san-francisco',
 '/biz/tadich-grill-san-francisco',
 '/biz/mahalo-bowl-sunnyvale-2',
 '/biz/hinata-san-francisco',
 '/biz/apres-winter-lounge-east-palo-alto',
 '/biz/xiao-long-bao-kitchen-south-san-francisco',
 '/biz/benkyodo-co-san-francisco',
 '/biz/mission-pie-san-francisco',
 '/biz/neighbor-bakehouse-san-francisco',
 '/biz/ginto-izakaya-japonaise-san-francisco',
 '/biz/chile-pies-baking-co-san-francisco',
 '/biz/goc

In [81]:
my_bm_details = []

for business in my_bm_df['business_id']:
    df = get_biz_details(api_key, business)
    my_bm_details.append(df)

In [85]:
my_bm_details_df = []

for business in my_bm_details:
    my_bm_details_df.append(json_to_df(business, details))
    

In [93]:
my_bm_det_df = pd.concat(my_bm_details_df)

In [94]:
my_bm_det_df

,alias,cat_0,cat_1,cat_2,is_closed,price,rating,review_count
0,three-twins-ice-cream-san-francisco-5,icecream,desserts,NaN,True,$,4.0,149.0
0,ben-thai-cafe-san-francisco,thai,NaN,NaN,False,$$,4.5,368.0
0,ijji-sushi-san-francisco,sushi,japanese,NaN,False,$$$$,4.5,162.0
0,koi-palace-daly-city,dimsum,seafood,cantonese,False,$$,3.5,3003.0
0,robin-san-francisco,sushi,NaN,NaN,False,$$$$,4.0,245.0
0,maria-catita-restaurante-e-loja-regional-lisboa,portuguese,NaN,NaN,False,€€,4.0,23.0
0,juns-beauty-salon-san-francisco,hair,skincare,threadingservices,False,$$,4.5,369.0
0,bowld-acai-san-francisco-2,foodtrucks,hawaiian,acaibowls,False,$$,4.5,525.0
0,china-bee-san-mateo,chinese,taiwanese,NaN,False,$,4.0,720.0
0,bobos-san-francisco-14,steak,seafood,NaN,False,$$$,4.0,1961.0


In [13]:
# create dataframe for my own bookmarks
my_bm_df = pd.DataFrame({'url': self_bookmarks,
                         'id' : self_id,
                         'business_id' : [bm.split('/biz/')[1] for bm in self_bookmarks]})

my_bm_df

,business_id,id,url
0,three-twins-ice-cream-san-francisco-5,ShHBKjuJbQAVBLs7DgA95A,/biz/three-twins-ice-cream-san-francisco-5
1,ben-thai-cafe-san-francisco,ShHBKjuJbQAVBLs7DgA95A,/biz/ben-thai-cafe-san-francisco
2,ijji-sushi-san-francisco,ShHBKjuJbQAVBLs7DgA95A,/biz/ijji-sushi-san-francisco
3,koi-palace-daly-city,ShHBKjuJbQAVBLs7DgA95A,/biz/koi-palace-daly-city
4,robin-san-francisco,ShHBKjuJbQAVBLs7DgA95A,/biz/robin-san-francisco
5,maria-catita-restaurante-e-loja-regional-lisboa,ShHBKjuJbQAVBLs7DgA95A,/biz/maria-catita-restaurante-e-loja-regional-...
6,juns-beauty-salon-san-francisco,ShHBKjuJbQAVBLs7DgA95A,/biz/juns-beauty-salon-san-francisco
7,bowld-acai-san-francisco-2,ShHBKjuJbQAVBLs7DgA95A,/biz/bowld-acai-san-francisco-2
8,china-bee-san-mateo,ShHBKjuJbQAVBLs7DgA95A,/biz/china-bee-san-mateo
9,bobos-san-francisco-14,ShHBKjuJbQAVBLs7DgA95A,/biz/bobos-san-francisco-14


In [104]:
friend_bm_df = [create_df_bookmarks(key, friend_bookmarks) for key in friend_bookmarks.keys()]
friend_bm_df = pd.concat(friend_bm_df)
friend_bm_df['business_id'] = friend_bm_df['url'].apply(lambda x: split_biz_url(x))
friend_bm_df

,url,id,business_id
0,NaN,T1ZDOuZxeBK-SaowZqYl_w,None
0,/biz/comoncy-scottsdale-4,7IV3JXTXufasWNSkDEdrcQ,comoncy-scottsdale-4
1,/biz/padrecito-san-francisco,7IV3JXTXufasWNSkDEdrcQ,padrecito-san-francisco
2,/biz/everett-and-jones-barbeque-oakland-4,7IV3JXTXufasWNSkDEdrcQ,everett-and-jones-barbeque-oakland-4
3,/biz/merrimans-kapalua-kapalua,7IV3JXTXufasWNSkDEdrcQ,merrimans-kapalua-kapalua
4,/biz/aa-roots-lahaina-5,7IV3JXTXufasWNSkDEdrcQ,aa-roots-lahaina-5
5,/biz/sasaki-san-francisco-2,7IV3JXTXufasWNSkDEdrcQ,sasaki-san-francisco-2
6,/biz/del-mar-san-francisco-3,7IV3JXTXufasWNSkDEdrcQ,del-mar-san-francisco-3
7,/biz/stonemill-matcha-san-francisco,7IV3JXTXufasWNSkDEdrcQ,stonemill-matcha-san-francisco
8,/biz/the-lot-sf-san-francisco-3,7IV3JXTXufasWNSkDEdrcQ,the-lot-sf-san-francisco-3


In [ ]:
friend_bm_det_df = api_to_df(api_key, business, details, df)

# Get Business Details from Reviews

In [99]:
# load scraped data
my_reviews = load_obj('my_reviews')
friend_reviews = load_obj('friends_reviews')

In [110]:
my_review_df = pd.DataFrame(list(my_reviews.items()), columns=['url', 'rating'])
my_review_df['id'] = self_id
my_review_df['business_id'] = my_review_df['url'].apply(lambda x: split_biz_url(x))
my_review_df

,url,rating,id,business_id
0,/biz/prubechu-san-francisco,5,ShHBKjuJbQAVBLs7DgA95A,prubechu-san-francisco
1,/biz/em-lash-and-brow-san-jose,5,ShHBKjuJbQAVBLs7DgA95A,em-lash-and-brow-san-jose
2,/biz/tora-ya-ramen-ville-de-qu%C3%A9bec,3,ShHBKjuJbQAVBLs7DgA95A,tora-ya-ramen-ville-de-qu%C3%A9bec
3,/biz/fuji-sukiyaki-san-mateo,3,ShHBKjuJbQAVBLs7DgA95A,fuji-sukiyaki-san-mateo
4,/biz/ristorante-quattro-montr%C3%A9al-2,5,ShHBKjuJbQAVBLs7DgA95A,ristorante-quattro-montr%C3%A9al-2
5,/biz/milkweed-boston,4,ShHBKjuJbQAVBLs7DgA95A,milkweed-boston
6,/biz/lukes-lobster-downtown-crossing-boston,5,ShHBKjuJbQAVBLs7DgA95A,lukes-lobster-downtown-crossing-boston
7,/biz/il-fornaio-santa-clara-2,3,ShHBKjuJbQAVBLs7DgA95A,il-fornaio-santa-clara-2
8,/biz/larb-thai-food-and-tapas-el-cerrito,5,ShHBKjuJbQAVBLs7DgA95A,larb-thai-food-and-tapas-el-cerrito
9,/biz/sprint-store-san-jose-8,1,ShHBKjuJbQAVBLs7DgA95A,sprint-store-san-jose-8


In [111]:
my_rev_det_df = api_to_df(api_key, details, my_review_df)

In [112]:
my_rev_det_df

,alias,cat_0,cat_1,cat_2,is_closed,price,rating,review_count
0,prubechu-san-francisco,guamanian,NaN,NaN,False,$$$,4.5,332.0
0,em-lash-and-brow-san-jose,eyelashservice,permanentmakeup,eyebrowservices,False,$$,5.0,32.0
0,tora-ya-ramen-ville-de-québec,ramen,NaN,NaN,False,$$,4.5,28.0
0,fuji-sukiyaki-san-mateo,japanese,sushi,NaN,False,$$,4.0,440.0
0,ristorante-quattro-montréal-2,italian,NaN,NaN,False,$$$,4.0,108.0
0,milkweed-boston,newamerican,breakfast_brunch,cocktailbars,False,$$,4.5,250.0
0,lukes-lobster-downtown-crossing-boston,seafood,NaN,NaN,False,$$,4.5,320.0
0,il-fornaio-santa-clara-2,pizza,seafood,italian,False,$$,4.5,875.0
0,larb-thai-food-and-tapas-el-cerrito,thai,NaN,NaN,False,$$,4.5,327.0
0,sprint-store-san-jose-8,mobilephones,electronics,cellphoneaccessories,False,$$$,2.0,173.0
